## FFM Application with Python

### 1. Create Test Signals
To study how FFT works, I first created code to generate multiple sine signals using the following general form of a sine wave.
$$y = A \sin(2\pi f t + \phi)$$

In [186]:
import numpy as np
import os
import librosa
import IPython.display as ipd 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [187]:
class sinWave:
    def __init__(self, **kwargs):
        self.A = kwargs.get('amp', 1)
        self.f = kwargs.get('freq', 1)
        self.p = kwargs.get('phase_shift', 0)
        self.endTime = kwargs.get('endTime', 1)
        self.sampleTime = kwargs.get('sampleTime', 0.01)

    def getDomain(self):
        return np.arange(0.0, self.endTime, self.sampleTime)
        
    def createSinWave(self,time):        
        return self.A * np.sin(2*np.pi*self.f*time + self.p)

The following code generate three different sine waves, 

$2sin(2\pi)$,

$5sin(3\cdot2\pi)$, and

$9sin(2\cdot2\pi)$.

In [274]:
sr = 1/22050 #standard 44.1 kHz
amplitude = 1
a4_freq = 440 
csharp5_freq = 554.365
e5_freq = 659.365

In [275]:
# generate sin waves
sin = sinWave(amp = amplitude, sampleTime=sr)
t = sin.getDomain()
sin.f = a4_freq
a4_y = sin.createSinWave(time=t)
sin.f = csharp5_freq
csharp5_y = sin.createSinWave(time=t)
sin.f = e5_freq
e5_y = sin.createSinWave(time=t)
a_chord = a4_y + csharp5_y + e5_y

#plot graph
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=t[:len(t)//10],
    y=a4_y[:len(t)//10],
    mode='lines',
    name='A4',
    line=dict(color='rgba(173, 216, 230, 0.6)'), 
))

fig.add_trace(go.Scatter(
    x=t[:len(t)//10],
    y=csharp5_y[:len(t)//10],
    mode='lines',
    name='C#5',
    line=dict(color='rgba(144, 238, 144, 0.6)'),  
))

fig.add_trace(go.Scatter(
    x=t[:len(t)//10],
    y=e5_y[:len(t)//10],
    mode='lines',
    name='E5',
    line=dict(color='rgba(255, 182, 193, 0.6) '), 
))

fig.add_trace(go.Scatter(
    x=t[:len(t)//10],
    y=a_chord[:len(t)//10],
    mode='lines',
    name='A Chord',
    line=dict(color='rgba(221, 160, 221, 1.0)') 
))

fig.update_layout(
    title={
        'text':'Original, Noisy, and Filtered Signal',
        'x': 0.5},
    xaxis_title='Time (s)',
    yaxis_title='Amplitude',
    font=dict(family="Arial", size=14)
)

fig.show()

In [276]:
a4_audio = ipd.Audio(data=a4_y, rate=1/sr)
csharp5_audio = ipd.Audio(data=csharp5_y, rate=1/sr)
e5_audio = ipd.Audio(data=e5_y, rate=1/sr)
ipd.Audio(data = a_chord, rate=1/sr)
noisy_a_chord.shape

(22050,)

In [277]:
noise_level = 3
noisy_a_chord = a_chord + noise_level * np.random.randn(len(a_chord))
sr = 1/22050 #standard 44.1 kHz
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=t[:len(t)//10],
    y=noisy_a_chord[:len(t)//10],
    mode='lines',
    name='noisy A chord',
    line=dict(color='rgba(221, 160, 221, 1.0)'), 
))
fig.add_trace(go.Scatter(
    x=t[:len(t)//10],
    y=a_chord[:len(t)//10],
    mode='lines',
    name='original A chord',
    line=dict(color='rgba(144, 238, 144, 0.8)'), 
))
fig.show()
ipd.Audio(data = noisy_a_chord, rate=1/sr)

In [278]:
N = len(a_chord)               # number of sampling points
sr = 1/sin.sampleTime    # sampling rate (# samples per second)
T = N/sr                            # total time
k = np.arange(N)                    # index for frequency [0-(n-1)]
freq = k/T                          # plotting points

In [279]:
a_chord_fft = np.fft.fft(a_chord)
a_chord_fft_ps = (np.abs(a_chord_fft)**2) * (1/N**2)
a_chord_fft_ps_one_sided = a_chord_fft_ps[:N//2].copy()

noisy_a_chord_fft = np.fft.fft(noisy_a_chord)
noisy_a_chord_fft_ps = (np.abs(noisy_a_chord_fft)**2) * (1/N**2)
noisy_a_chord_fft_ps_one_sided = noisy_a_chord_fft_ps[:N//2].copy()


In [280]:
graph_a_chord_fft = go.Figure()
graph_a_chord_fft.add_trace(go.Scatter(
    x=freq[:1000],
    y=np.real(np.abs(a_chord_fft[:1000])),
    mode='markers+lines',
    name='A chord FFT',
    line=dict(color='rgba(144, 238, 144, 0.5)'), 
))

graph_noisy_a_chord_fft = go.Figure()
graph_noisy_a_chord_fft.add_trace(go.Scatter(
    x=freq[:1000],
    y=np.real(np.abs(noisy_a_chord_fft[:1000])),
    mode='markers+lines',
    name='noisy A chord FFT',
    line=dict(color='rgba(221, 160, 221, 1.0)'), 
))

combined_graphs = make_subplots(rows=1, cols=2, subplot_titles=('A Chord FFT', 'Noisy A Chord FFT'))
combined_graphs.add_traces(graph_a_chord_fft.data, rows=1, cols=1)
combined_graphs.add_traces(graph_noisy_a_chord_fft.data, rows=1, cols=2)

In [ ]:
full_mask = abs(noisy_a_chord_fft) > 1500
fhat = full_mask * noisy_a_chord_fft

graph_denoised_a_chord_fft = go.Figure()
graph_denoised_a_chord_fft.add_trace(go.Scatter(
    x=freq,
    y=np.real(np.abs(fhat)),
    mode='markers+lines',
    name='A chord FFT',
    line=dict(color='rgba(144, 238, 144, 0.5)'), 
))
graph_denoised_a_chord_fft.show()
denoised_achord_ifft = np.fft.ifft(fhat)

graph_denoised_a_chord_ifft = go.Figure()
graph_denoised_a_chord_ifft.add_trace(go.Scatter(
    x=t[:1000],
    y=abs(denoised_achord_ifft[:1000]),
    mode='markers+lines',
    name='A chord FFT',
    line=dict(color='rgba(144, 238, 144, 0.5)'), 
))
# threshold = 10
# noisy_a_chord_fft = np.fft.fft(noisy_a_chord)
# noisy_a_chord_fft[np.abs(noisy_a_chord_fft) < threshold] = 0
# denoised_achord_ifft = np.real(np.fft.ifft(noisy_a_chord_fft))
# graph_denoised_a_chord_ifft = go.Figure()
# graph_denoised_a_chord_ifft.add_trace(go.Scatter(
#     x=t[:1000],
#     y=abs(denoised_achord_ifft[:1000]),
#     mode='markers+lines',
#     name='A chord FFT',
#     line=dict(color='rgba(144, 238, 144, 0.5)'), 
# ))
